In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import os
import pandas as pd
import cv2

In [2]:
# --- Configuration ---
preprocessed_path = r'D:\FUCK!!\Pattern\Project\notebooks\new_preprocessed_RGB_images' # Make sure this path is correct
input_shape = (224, 224, 3)  # Standard VGG16 RGB input
batch_size = 64
num_epochs = 20 # Set a higher number of epochs, EarlyStopping will handle the actual stop

In [3]:
from tensorflow.keras.applications.vgg16 import preprocess_input

# “val_dir” is the root of your test/validation folder
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

print("Setting up Training Generator...")
train_generator = train_datagen.flow_from_directory(
    os.path.join(preprocessed_path, 'train'),
    target_size=input_shape[:2], # Use (224, 224)
    batch_size=batch_size,
    class_mode='sparse',         # Use sparse for sparse_categorical_crossentropy
    color_mode='rgb',
    shuffle=True             # Ensure images are loaded as RGB
)
print(f"Found {train_generator.samples} images belonging to {train_generator.num_classes} classes in training set.")

print("\nSetting up Validation Generator...")
val_generator = train_datagen.flow_from_directory(
    os.path.join(preprocessed_path, 'val'),
    target_size=input_shape[:2], # Use (224, 224)
    batch_size=batch_size,
    class_mode='sparse',         # Use sparse for sparse_categorical_crossentropy
    color_mode='rgb',            # Ensure images are loaded as RGB
    shuffle=False                # No need to shuffle validation data
)
print(f"Found {val_generator.samples} images belonging to {val_generator.num_classes} classes in validation set.")

print("\nSetting up Testing Generator...")
test_generator = train_datagen.flow_from_directory(
    r"D:\FUCK!!\Pattern\Project\notebooks\new_preprocessed_RGB_images\test",
    target_size=input_shape[:2], # Use (224, 224)
    batch_size=batch_size,
    class_mode='sparse',         # Use sparse for sparse_categorical_crossentropy
    color_mode='rgb',            # Ensure images are loaded as RGB
    shuffle=False                # No need to shuffle validation data
)
print(f"Found {test_generator.samples} images belonging to {test_generator.num_classes} classes in validation set.")


Setting up Training Generator...
Found 28400 images belonging to 10 classes.
Found 28400 images belonging to 10 classes in training set.

Setting up Validation Generator...
Found 497 images belonging to 10 classes.
Found 497 images belonging to 10 classes in validation set.

Setting up Testing Generator...
Found 268 images belonging to 10 classes.
Found 268 images belonging to 10 classes in validation set.


In [4]:
# Get the number of classes from the generator
num_classes = train_generator.num_classes
if num_classes != val_generator.num_classes:
    print("Warning: Training and validation sets have different numbers of classes!")


In [5]:
# --- Build VGG16 Model ---
print("\nBuilding VGG16 model...")
# Load VGG16 base pre-trained on ImageNet, exclude top classification layer
base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

# 2) Freeze early blocks, fine-tune later ones
for layer in base_model.layers[:-3]:
    layer.trainable = False
print(f"VGG16 base model loaded. Trainable: {base_model.trainable}")

# Create the new model head
vgg16_model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(name='global_avg_pool'),
    layers.BatchNormalization(),
    layers.Dense(512, activation='relu', name='fc1'),
    layers.Dropout(0.75, name='dropout_1'), # Dropout for regularization
    layers.Dense(num_classes, activation='softmax', name='predictions') # Output layer
], name="VGG16_Transfer_Learning")

# --- Compile Model ---
# Use Adam optimizer with a specific, smaller learning rate
optimizer = Adam(learning_rate=0.0001) # 0.0001 learning rate

vgg16_model.compile(optimizer=optimizer,
                    loss='sparse_categorical_crossentropy', # Use sparse CE with class_mode='sparse'
                    metrics=['accuracy'])

print("\nModel Summary:")
vgg16_model.summary()


Building VGG16 model...
VGG16 base model loaded. Trainable: True

Model Summary:


Model: "VGG16_Transfer_Learning"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_avg_pool                 │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ predictions (Dense)             │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,984,522 (57.16 MB)

 Trainable params: 4,988,426 (19.03 MB)

 Non-trainable params: 9,996,096 (38.13 MB)

In [6]:
# --- Define Callbacks ---
# Early stopping to prevent overfitting and stop training when val_loss stops improving
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,          # Number of epochs with no improvement after which training will be stopped
    verbose=1,
    restore_best_weights=True # Restore model weights from the epoch with the best val_loss
)

# Model checkpoint to save the best model found during training
model_checkpoint = ModelCheckpoint(
    filepath='best_vgg16_model.keras', # File path to save the model
    monitor='val_loss',
    save_best_only=True, # Only save a model if `val_loss` has improved
    verbose=1
)

In [10]:
# --- Train the Model (Head Only) ---
print(f"\nStarting training for up to {num_epochs} epochs (head only)...")
history = vgg16_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size, # Ensure steps cover the dataset
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size, # Ensure steps cover the dataset
    epochs=num_epochs,
    callbacks=[early_stopping, model_checkpoint] # Add the callbacks
)

print("\nTraining finished.")


Starting training for up to 20 epochs (head only)...


c:\Users\omarn\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
 40/443 ━━━━━━━━━━━━━━━━━━━━ 37:43 6s/step - accuracy: 0.1907 - loss: 2.9716

In [17]:
from tensorflow.keras.models import load_model
model = load_model(r'D:\FUCK!!\Pattern\Project\Models\best_vgg16_model.keras')

In [23]:
from tensorflow.keras.applications.vgg16 import preprocess_input
from sklearn.metrics import classification_report, confusion_matrix

# 1) Prepare a “pure” test generator (no augmentation, only the same preprocessing)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_gen = test_datagen.flow_from_directory(
    directory=r'D:/FUCK!!/Pattern/Project/notebooks/new_preprocessed_RGB_images/test',    # point this to your test‐folder 
    target_size=input_shape[:2],           # e.g. (224, 224)
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False                           # important! so labels/preds align
)

Found 268 images belonging to 10 classes.


In [ ]:
# 0) Clear any old graphs / functions
tf.keras.backend.clear_session()

# 1) Reload your .keras model
model = load_model(r'D:\FUCK!!\Pattern\Project\Models\best_vgg16_model.keras')

# 2) Re-compile it with the correct loss and metrics, running eagerly
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
    run_eagerly=True    # ← this disables the tf.function wrapping
)

# 3) Now evaluate on your test generator (no need to pass steps)
loss, acc = model.evaluate(test_gen, verbose=1)
print(f"\nTest loss: {loss:.4f} — Test accuracy: {acc:.4f}")


5/5 ━━━━━━━━━━━━━━━━━━━━ 17s 3s/step - accuracy: 0.6265 - loss: 26.7120

Test loss: 33.0836 — Test accuracy: 0.5560


In [ ]:
# 3) (Optional) Get per‐class metrics
#    - Predict class probabilities
steps = int(np.ceil(test_gen.samples / batch_size))
pred_probs = model.predict(
    test_gen,
    steps = steps,
    verbose=1
)
#    - Turn them into predicted class indices
pred_idxs = np.argmax(pred_probs, axis=1)
true_idxs = test_gen.classes
labels   = list(test_gen.class_indices.keys())

#    - Classification report
print("\nClassification Report:\n")
print(classification_report(true_idxs, pred_idxs, target_names=labels))

#    - Confusion matrix
print("\nConfusion Matrix:\n")
print(confusion_matrix(true_idxs, pred_idxs))

5/5 ━━━━━━━━━━━━━━━━━━━━ 16s 3s/step

Classification Report:

                                  precision    recall  f1-score   support

         Achaemenid architecture       0.78      0.80      0.79        40
American Foursquare architecture       0.86      0.84      0.85        37
        American craftsman style       0.56      0.67      0.61        21
   Ancient Egyptian architecture       0.00      0.00      0.00        18
           Art Deco architecture       0.67      0.25      0.36        24
        Art Nouveau architecture       0.53      0.50      0.51        20
            Baroque architecture       0.42      0.89      0.57        27
            Bauhaus architecture       0.36      1.00      0.53        27
         Beaux-Arts architecture       1.00      0.07      0.13        29
          Byzantine architecture       1.00      0.12      0.21        25

                        accuracy                           0.56       268
                       macro avg       0.62     

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='train_loss')
plt.plot(history.history['val_accuracy'], label='val_loss')
plt.legend(); plt.show()

NameError: name 'history' is not defined

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend(); plt.show()


NameError: name 'history' is not defined